# Code preparation

In [1]:
# colab_LDA/NMF_runner v1.1

# fit LDA/NMF with crisis_12
# Added CTM
# Finished

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install contractions
!pip install octis
#!pip install bertopic
#!pip install top2vec
#!pip install scikit_learn==1.0.2
#!pip install datashader
#!pip install symspellpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
%cd /content/drive/MyDrive/SS_2022_Praktikum

/content/drive/MyDrive/SS_2022_Praktikum


In [5]:
import os
import sys
import importlib
import nltk
import pandas as pd

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
# Should I in src?
assert os.path.exists('src'), f"[ERROR] The path src not detected in the parent directory '{os.getcwd()}'."

if os.getcwd().endswith('/notebooks'):
    os.chdir('..')
    sys.path.append('./src')

print(f'[INFO] Current Directory: "{os.getcwd()}".')

[INFO] Current Directory: "/content/drive/MyDrive/SS_2022_Praktikum".


# Training

In [7]:
configs = {
    'dataset': 'crisis_07',
    'preprocessing_funcs': [
        'to_lowercase',
        'standardize_accented_chars',
        'remove_url',
        'expand_contractions',
        'remove_mentions',
        'remove_hashtags',
        # 'remove_new_lines',
        'keep_only_alphabet',
        # 'remove_extra_spaces',
        'remove_english_stop_words',
        'lemmatize_noun'
    ],
    'algorithm': 'ctm',
    'algorithm_args': {
        'num_topics': 2,
        'model_name': 'ctm',     # Specific for LDA/NMF
        'data_save_dir': './OCTIS_data_tmp',   # Specific for LDA/NMF
        'random_state': 100,    # Specific for LDA/NMF
        # 'embedding_model': 'doc2vec',
        # 'embedding_model': 'universal-sentence-encoder',
        # 'embedding_model': 'universal-sentence-encoder-large', # WORKS VERY WELL
        # 'embedding_model': 'distiluse-base-multilingual-cased',
        #'doc2vec_speed': 'learn',
        #'min_count': 50,
        #'umap_args': {
        #    'n_neighbors': 15,
        #    'n_components': 5,
        #    'metric': 'cosine',
        #    'random_state': 42  # Try to always include this for reproducibility, github.com/ddangelov/Top2Vec/issues/86
        },
        #'hdbscan_args': {
        #    'min_cluster_size': 15,
        #    'metric': 'euclidean',
        #    'cluster_selection_method': 'eom'
        #}
    }

In [8]:
from src import preprocessor
from src.utils import load_documents

docs,labels = load_documents(dataset=configs['dataset'])
if 'preprocessing_funcs' in configs:
    docs = preprocessor.run(data=docs, prep_functions=configs['preprocessing_funcs'])
    
algorithm_args = configs['algorithm_args']
algorithm_args.update(data_name=configs['dataset'],docs=docs,labels=labels)

algorithm_name = configs['algorithm'].lower()
if algorithm_name == 'lda' or 'nmf':
    from src import LDA_NMF_runner
    df_output_doc_topic, df_output_topic_word = LDA_NMF_runner.runner(args=algorithm_args)

[INFO] Available Preprocessing Functions in the Module:['to_lowercase', 'standardize_accented_chars', 'remove_url', 'expand_contractions', 'expand_missing_delimiter', 'remove_mentions', 'remove_hashtags', 'keep_only_alphabet', 'remove_new_lines', 'remove_extra_spaces', 'remove_html_tags', 'remove_english_stop_words', 'lemmatize', 'lemmatize_verb', 'lemmatize_noun', 'lemmatize_adjective', 'correct_typo']
[INFO] Preprocessing starting..
[INFO] These string preprocessing methods will be applied to the data in order:
(  'to_lowercase',
   'standardize_accented_chars',
   'remove_url',
   'expand_contractions',
   'remove_mentions',
   'remove_hashtags',
   'keep_only_alphabet')
[INFO] Then, these tokenized preprocessing methods will be applied to the data in order:
(  '__tokenize',
   'remove_english_stop_words',
   'lemmatize_noun',
   '__glue')
[INFO] Preprocessing completed in 8.643 seconds..


Batches:   0%|          | 0/81 [00:00<?, ?it/s]

Batches:   0%|          | 0/11 [00:00<?, ?it/s]

Batches:   0%|          | 0/11 [00:00<?, ?it/s]

In [9]:
# See doc_topic_matrix
df_output_doc_topic

,run_id,Document ID,Document,Real Label,Assigned Topic Num,Assignment Score
0,1658351281,0,rt villager district desperate help many broken,earthquake,0,0.717289
1,1658351281,1,rt provide clean drinking water family please ...,earthquake,1,0.606577
2,1658351281,2,queensland flood crisis deepens death toll ris...,flood,1,0.963342
3,1658351281,3,nepal situation day quake pretty bad death tol...,earthquake,1,0.584877
4,1658351281,4,rt flash flood australia australia edis number...,flood,1,0.604373
...,...,...,...,...,...,...
8082,1658351281,8082,rt iaf today evacuated casualty amp person cas...,earthquake,0,0.611530
8083,1658351281,8083,hi bay could give follow back speak nepal amp ...,earthquake,0,0.707407
8084,1658351281,8084,queensland flood crisis deepens australian,flood,1,0.932460
8085,1658351281,8085,latest news queensland flood crisis deepens au...,flood,1,0.942468


In [10]:
# See Topic_Word_matrix
df_output_topic_word

,run_id,method,method_specific_params,dataset,num_given_topics,reduced,topic_num,topic_size,topic_words,word_scores,num_detected_topics,num_final_topics,duration_secs
0,1658351281,ctm,None,crisis_07,2,None,0,5000,"[amp, quake, please, look, claim, weather, hel...","[0.113775365, 0.11029904, 0.10864016, 0.104650...",None,None,506.76
1,1658351281,ctm,None,crisis_07,2,None,1,3087,"[flood, queensland, rise, australia, crisis, n...","[0.16751276, 0.14718118, 0.14072692, 0.1284164...",None,None,506.76


# Visualization

In [11]:
import importlib
from src import visualizer; importlib.reload(visualizer)

<module 'src.visualizer' from '/content/drive/MyDrive/SS_2022_Praktikum/src/visualizer.py'>

In [12]:
visualizer.visualize_labels_per_topic(df_output_doc_topic = df_output_doc_topic,
                                      df_output_topic_word = df_output_topic_word,
                                      top_n_topics = algorithm_args['num_topics'])

AssertionError: ignored

In [ ]:
visualizer.visualize_barchart(df_output_topic_word = df_output_topic_word,n_words = 5)

In [ ]:
# First Documents
for i in sorted(df_output_topic_word['topic_num'].to_list()):
    x = df_output_doc_topic.query(f'`Assigned Topic Num` == {i}').sort_values(by ='Assignment Score',ascending=False).drop('run_id',axis=1).head()
    with pd.option_context('display.max_colwidth', None):
        display(x)

In [ ]:
assert False

In [ ]:
a = [0.2, 0.3]
import numpy as np
a /= 0.5
a

In [ ]:
from gensim.corpora import Dictionary

documents = ["Human machine interface for lab abc computer applications",
             "A survey of user opinion of computer system response time",
             "The EPS user interface management system",
             "System and human system engineering testing of EPS",
             "Relation of user perceived response time to error measurement",
             "The generation of random binary unordered trees",
             "The intersection graph of paths in trees",
             "Graph minors IV Widths of trees and well quasi ordering",
             "Graph minors A survey"]

dataset = []
for text in documents:
    tokens = nltk.word_tokenize(text)
    dataset.append(tokens)
vocab = Dictionary(documents=dataset, prune_at=20000)

In [ ]:
vocab.filter_extremes(no_below=1, no_above=1, keep_n=10) 

In [ ]:
word_list = []
for key in vocab.token2id.keys():
    word_list.append(key)
word_list

In [ ]:
# Then vocabulary.txt is obtained using the function we have

In [ ]:
for i in range(len(documents)):
    new_sen = []
    for word in nltk.word_tokenize(documents[i]):
        if word in word_list:
            new_sen.append(word)
    documents[i] = ' '.join(new_sen).strip()

In [ ]:
documents

In [ ]:
vocab.token2id

In [ ]:
vocab.id2token

In [ ]:
vocab.cfs

In [ ]:
vocab.filter_extremes(no_below=2)

# Archive

In [ ]:
assert False

In [ ]:
# In Archive
from src import LDA_NMF_runner
importlib.reload(LDA_NMF_runner)

# Please choose the entries from the following sets:

# model: ['LDA', 'NMF']
# dataset: ['crisis_12', '20news']

# Other parameters
# top_n_topics: int (This decides how many topics are shown)
# n_words: int (This decides how many words are shown)

settings = {'model': 'NMF',
            'dataset': 'crisis_12',
            'top_n_topics': 5,
            'n_words': 5}

save_dir_crisis_12 = '/content/drive/MyDrive/SS_2022_Praktikum/Crisis Dataset/Dataset_12'

Doc_Topic_df, Topic_Word_df = LDA_NMF_runner.runner(model_name = settings['model'],
                                                    dataset_name = settings['dataset'],
                                                    top_n_topics = settings['top_n_topics'],
                                                    n_words = settings['n_words'],
                                                    save_dir = save_dir_crisis_12,
                                                    random_state = 100)